In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType

spark = (
    SparkSession.builder
        .appName("cdc")
        .getOrCreate()
)

/usr/local/lib/python3.8/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 15:25:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Intial full load
df_fl = spark.read.csv("../data/LOAD00000001.csv")

df_fl = (
    df_fl
    .withColumnRenamed("_c0", "id")
    .withColumnRenamed("_c1", "fullName")
    .withColumnRenamed("_c2", "City")
)

df_fl.write.mode("overwrite").csv("../data/output/final_file.csv", header=True)
df_fl.show(3)


# Updates to make on the full load
df_update = spark.read.csv("../data/20231212-135710944.csv")
df_update = (
    df_update
    .withColumnRenamed("_c0", "action")
    .withColumnRenamed("_c1", "id")
    .withColumnRenamed("_c2", "fullName")
    .withColumnRenamed("_c3", "City")
)

# Load full data and apply updatesa
df_ff = spark.read.options(
    inferSchema=True,
    header=True
).csv("../data/output/final_file.csv")


for row in df_update.collect():

    act = row["action"]

    if act == "U":
        df_ff = df_ff.withColumn("FullName", F.when(df_ff["id"] == row["id"], row["fullName"]).otherwise(df_ff["FullName"]))      
        df_ff = df_ff.withColumn("City", F.when(df_ff["id"] == row["id"], row["City"]).otherwise(df_ff["City"]))
    elif act == "I":
        row_to_insert = [list(row)[1:]]
        df_ff = df_ff.union(spark.createDataFrame(row_to_insert, df_ff.columns))
    elif act == "D":
        df_ff = df_ff.filter(df_ff.id != row["id"])


df_ff.show()